In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from scripts import calculate_runtime_pytorch

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

Using cpu device


In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
def run():
    epochs = 2
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
    print("Done!")

In [9]:
calculate_runtime_pytorch(fct=run, device=device)

Epoch 1
-------------------------------
loss: 2.296156  [    0/60000]
loss: 2.285780  [ 6400/60000]
loss: 2.265970  [12800/60000]
loss: 2.257548  [19200/60000]
loss: 2.244910  [25600/60000]
loss: 2.216799  [32000/60000]
loss: 2.219127  [38400/60000]
loss: 2.182152  [44800/60000]
loss: 2.169382  [51200/60000]
loss: 2.151394  [57600/60000]
Test Error: 
 Accuracy: 56.6%, Avg loss: 2.140616 

Epoch 2
-------------------------------
loss: 2.151196  [    0/60000]
loss: 2.140200  [ 6400/60000]
loss: 2.076729  [12800/60000]
loss: 2.087958  [19200/60000]
loss: 2.042326  [25600/60000]
loss: 1.988470  [32000/60000]
loss: 2.001377  [38400/60000]
loss: 1.921269  [44800/60000]
loss: 1.918373  [51200/60000]
loss: 1.859824  [57600/60000]
Test Error: 
 Accuracy: 60.0%, Avg loss: 1.852506 

Done!


Total average CPU runtime: 21.242 seconds

In [10]:
calculate_runtime_pytorch(fct=run, device=device, profiler=True)

Epoch 1
-------------------------------
loss: 1.892306  [    0/60000]


STAGE:2022-12-01 13:38:15 67288:67288 ActivityProfilerController.cpp:294] Completed Stage: Warm Up


loss: 1.861963  [ 6400/60000]
loss: 1.734960  [12800/60000]
loss: 1.765979  [19200/60000]
loss: 1.678853  [25600/60000]
loss: 1.633060  [32000/60000]
loss: 1.639533  [38400/60000]
loss: 1.544527  [44800/60000]
loss: 1.570843  [51200/60000]
loss: 1.472126  [57600/60000]
Test Error: 
 Accuracy: 60.6%, Avg loss: 1.490620 

Epoch 2
-------------------------------
loss: 1.565379  [    0/60000]
loss: 1.533720  [ 6400/60000]
loss: 1.376837  [12800/60000]
loss: 1.438991  [19200/60000]
loss: 1.343883  [25600/60000]
loss: 1.334441  [32000/60000]
loss: 1.340371  [38400/60000]
loss: 1.266332  [44800/60000]
loss: 1.305535  [51200/60000]
loss: 1.208375  [57600/60000]
Test Error: 
 Accuracy: 62.8%, Avg loss: 1.236997 

Done!


STAGE:2022-12-01 13:38:48 67288:67288 ActivityProfilerController.cpp:300] Completed Stage: Collection


-------------------------------------------------------  ------------  ------------  ------------  ------------  
------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  
CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  
------------  ------------  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        54.11%       15.948s        74.28%       21.891s    
9.977ms          2194  
                                               aten::mm         8.26%        2.433s         8.26%        2.433s    
259.382us          9380  
                                            aten::addmm         5.88%        1.734s         6.22%        1.832s    
278.789us          6570  
                                             aten::add_         4.47%        1.318s         4.47%        1.318s    
58.562us         22512  
                                              aten::div         3.48%        1.026s         6.02%        1.775s    
12.678us        140000  
                                         aten::_to_copy         3.34%     983.220ms         6.70%        1.975s    
7.047us        280314  
                                            aten::copy_         2.55%     750.369ms         2.55%     750.369ms    
2.616us        286884  
                                           aten::select         2.53%     746.266ms         2.59%     763.073ms    
2.725us        280000  
                                          aten::permute         1.90%     560.895ms         1.97%     579.571ms    
4.140us        140000  
                                            aten::fill_         1.39%     409.338ms         1.39%     409.338ms    
35.379us         11570  
                                               aten::to         1.29%     381.317ms         7.72%        2.276s    
5.306us        429074  
                                             aten::view         1.19%     349.285ms         1.19%     349.285ms    
2.363us        147818  
                                    aten::empty_strided         1.18%     347.098ms         1.18%     347.098ms    
1.230us        282190  
                                                 detach         0.99%     290.682ms         0.99%     290.682ms    
2.076us        140000  
                                Optimizer.step#SGD.step         0.79%     232.201ms         3.42%        1.009s    
537.894us          1876  
                                           aten::detach         0.64%     188.955ms         1.57%     463.992ms    
3.314us        140000  
                                             aten::item         0.54%     158.977ms         0.57%     167.137ms    
1.188us        140652  
                                              aten::cat         0.46%     135.508ms         0.51%     151.071ms    
68.982us          2190  
                                              aten::sum         0.42%     122.669ms         0.45%     132.999ms    
22.342us          5953  
                               aten::threshold_backward         0.39%     113.842ms         0.39%     113.842ms    
30.342us          3752  
                                                aten::t         0.35%     104.529ms         0.62%     183.675ms    
6.751us         27206  
                      Optimizer.zero_grad#SGD.zero_grad         0.34%      99.703ms         1.76%     518.787ms    
276.539us          1876  
                                        aten::clamp_min         0.31%      91.743ms         0.31%      91.743ms    
20.946us          4380  
    autograd::engine::evaluate_function: AddmmBackward0         0.31%      91.738ms         9.61%        2.833s    
503.341us          5628  
                                         AddmmBackward0         0.27%      78.436ms         8.80%        2.594s    
460.906us          5628  
                                       aten::as_stri

Total average CPU runtime: 0.013 seconds

Total average GPU runtime: 0.000 seconds